In [2]:
import numpy as np
import networkx as nx
from collections import defaultdict


# Fully connected problems

In [3]:
def WMaxCut(G):
    # Weighted maxcut problem from networkx graph model as a QUBO problem
    mdl = Model('MaxCut')
    x = {i: mdl.binary_var(name=f"x_{i}") for i in sorted(G.nodes())}
    mdl.minimize(-mdl.sum(G[i][j]["weight"]*(x[i] * (1 - x[j]) + x[j] * (1 - x[i]))  for (i, j) in G.edges))
    return mdl

# def WMaxCut(G):
#     # Weighted maxcut problem from networkx graph model as a QUBO problem
#     mdl = Model('MaxCut')
#     num_vertices = G.number_of_nodes()
#     translate = {j:i for i, j in enumerate(sorted(G.nodes))}
#     x = {i: mdl.binary_var(name=f"x_{i}") for i in range(num_vertices)}
#     mdl.minimize(-mdl.sum(G[i][j]["weight"]*(x[translate[i]] * (1 - x[translate[j]]) + x[translate[j]] * (1 - x[translate[i]]))  for (i, j) in G.edges))
#     return mdl

In [4]:
#np.random.seed(1)
#problems = defaultdict(dict)
#
#num_qubits = range(10, 20)
#for nq in num_qubits:
#    print(f"Number of qubits: {nq}")
#    G  = nx.Graph()
#    G.add_nodes_from(range(nq))
#    for i in range(nq):
#        for j in range(i + 1, nq):
#            w = np.random.choice([0.1, 0.2, 0.5, 2.0])
#            G.add_weighted_edges_from([[i,j,w]])
#    problems[nq]["G"] = G
#    mdl = WMaxCut(G)
#    mdl.solve(time_limit=60) # Solving using CPLEX
#    problems[nq]["time_cplex"] = mdl.get_solve_details().deterministic_time
#    problems[nq]["real_time_cplex"] = mdl.get_solve_details().time
#    problems[nq]["n_iterations_cplex"] = mdl.get_solve_details().nb_iterations
#    problems[nq]["status"] = mdl.get_solve_details().status
#    problems[nq]["sol"] = "".join(str(round(mdl.solution.get_value(var))) for var in mdl.iter_binary_vars())
## np.save("./Data/WMC_FC.npy", problems)

# Native Layout Graph

In [5]:
#from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
#service = QiskitRuntimeService()

In [6]:
#backend_name = "ibm_fez"
#backend = service.backend(backend_name)

In [7]:
#np.random.seed(123)
#n_qubits = backend.num_qubits
#problem = {}
#G = nx.Graph()
#G.add_nodes_from(range(n_qubits))
#for i,j in backend.coupling_map.get_edges(): 
#    w = np.random.choice([0.1, 0.2, 0.3, 0.5, 1.0])
#    G.add_weighted_edges_from([[i, j, w]])
#
#mdl = WMaxCut(G)
#mdl.solve()
#problem["G"] = G
#problem["sol"] = "".join(str(round(mdl.solution.get_value(var))) for var in mdl.iter_binary_vars())
#np.save(f"./Data/WMC_NL_{backend}.npy", problem)

### AWS devices

In [8]:
#from braket.aws import AwsDevice, AwsQuantumTask
#from braket.devices import Devices, LocalSimulator
#from braket.circuits import Circuit
#backends = {}
#backends["iqm_garnet"] = AwsDevice(Devices.IQM.Garnet)
#backends["rigetti_ankaa_3"] = AwsDevice(Devices.Rigetti.Ankaa3)


In [7]:
problems = np.load("./Data/NL-problems.npy", allow_pickle=True).item()
np.random.seed(123)
backend_name = "rigetti_ankaa_3"
problem = {}
G_backend = backends[backend_name].topology_graph
n_qubits = G_backend.number_of_nodes()

G = nx.Graph()
dict_vars = {j:i for i, j in enumerate(sorted(G_backend.nodes))}
G.add_nodes_from(sorted(G_backend.nodes()))
for i,j in G_backend.edges(): 
    w = np.random.choice([0.1, 0.2, 0.3, 0.5, 1.0])
    G.add_weighted_edges_from([[i, j, w]])

mdl = WMaxCut(G)
mdl.solve()
problem["G"] = G
problem["sol"] = "".join(str(round(mdl.solution.get_value(var))) for var in mdl.iter_binary_vars())
problems[backend_name] = problem
np.save("./Data/NL-problems.npy", problems)

# 1D-Chain Graph

In [9]:
qubits_in_line = {}
qubits_in_line["Eagle"] = list(range(13,-1,-1)) + [14] + list(range(18,33)) + [36] + list(range(51,36,-1)) + [52] + list(range(56,71)) + [74] + list(range(89,74,-1)) + [90] + list(range(94,109)) + [112] + list(range(126,112,-1))
qubits_in_line["Heron-r1"] = list(range(13,-1,-1)) + [15] + list(range(19,34)) + [37] + list(range(52,37,-1)) + [53] + list(range(57,72)) + [75] + list(range(90,75,-1)) + [91] + list(range(95,110)) + [113] + list(range(128,114,-1))
qubits_in_line["Heron-r2"] = list(range(0,16)) + [19] + list(range(35,20,-1)) + [36] + list(range(41,56)) + [59] + list(range(75,60,-1)) + [76] + list(range(81,96)) + [99] + list(range(115,100,-1)) + [116] + list(range(121,124))

# 1D-Chain Graph - LABS

In [11]:
from get_objective import labs_true_optimal_energy

In [21]:
backend_name = "Eagle"
np.random.seed(123)
num_qubits = range(5, 10) # 110)
problems = defaultdict(dict)
for nq in num_qubits:
    #problems[nq]["G"] = G
    #mdl = WMaxCut(G)
    #mdl.solve()
    result = labs_true_optimal_energy(nq)  
    objective_value =result.energy
    problems["sol"][nq]= objective_value
np.save(f"./Data/LABS.npy", problems)

In [24]:
problems

defaultdict(dict, {'sol': {5: 2, 6: 7, 7: 3, 8: 8, 9: 12}})

In [23]:
problems = np.load("./Data/LABS.npy", allow_pickle=True).item()


In [14]:
stop

NameError: name 'stop' is not defined

# Removing mdl conflicting data

In [12]:
import os 
import numpy as np
folders = [f for f in os.listdir("./Data/") if os.path.isdir(os.path.join("./Data/", f)) and not f.startswith('.')]
for sub_folder in folders:
    folder_path = f"./Data/{sub_folder}/"
    files = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f)) and not f.startswith('.')]

    # nq = 156
    # backend_name = "ibm_fez"
    # layout = "HE"
    # extra = "_fractional"
    for file in files:
        if not os.path.isdir(os.path.join(folder_path, file)) and not file.startswith('.'):
            data = np.load(folder_path+file, allow_pickle=True).item()
            try:
                data.pop("mdl")
            except:
                pass
            np.save(folder_path+file, data)